In [ ]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from time import sleep

#Pergunta ao usuário qual site gostaria de realizar as análises dos relatórios
ativos = input('Escolha ativos da bolsa de valores que deseja identificar seus indicadores: ')

#Retira os espaços em branco dos ativos e deixa-os todos com letra maiúscula
ativos_formatados = [ativo.strip().upper() for ativo in ativos.split(",")] 
print(f'Seus ativos são: {ativos_formatados}')

# Configuração do WebDriver (Chrome neste caso)
#Criando um serviço responsável por instalar a versão do chrome driver de forma automática
service = Service(ChromeDriverManager().install())
#Executando este serviço
navegador = webdriver.Chrome(service=service)

#Entrando no site statusinvest
navegador.get('https://statusinvest.com.br/')
sleep(12)

#Clicando para fechar a propaganda
navegador.find_element(By.XPATH, '/html/body/div[19]/div/div/div[1]/button/i').click()


#Para ativo dentro da lista de ativos que o usuário digitou:
for ativo in ativos_formatados:
    
    #clicando na bússula de pesquisa
    navegador.find_element(By.XPATH, '//*[@id="main-nav-nav"]/div/div[2]/div/ul/li[2]/a/i').click()

    #Colocando o ativo dentro da barra de pesquisa
    navegador.find_element(By.XPATH, '//*[@id="main-search"]/div[1]/span[1]/input[2]').send_keys(ativo)

    sleep(3) #aguardando 3 segundos para clicar


    # Inputar uma validação de digitação incorreta do ativo.
    #Clicando no ativo desejado
    navegador.find_element(By.XPATH, '//*[@id="main-search"]/div[2]/div/div/a').click()




    if navegador.find_element(By.XPATH, '//*[@id="main-header"]/div[2]/div/div[1]/div/ol/li[2]/a/span').text == "Fundos Imobiliários":

        #Identificando os principais indicadores da página
        fundo_imobiliario = navegador.find_element(By.XPATH, '//*[@id="main-header"]/div[2]/div/div[1]/div/ol/li[2]/a/span').text
        dividend_yield = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[4]/div/div[1]/strong').text
        p_pv = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[5]/div/div[2]/div/div[1]/strong').text
        numero_cotistas = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[5]/div/div[6]/div/div[1]/strong').text
        valor_mercado = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[5]/div/div[2]/div/div[2]/span[2]').text
        valor_caixa = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[5]/div/div[3]/div/div[2]/span[2]').text
        valor_patrimonio_total = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[5]/div/div[1]/div/div[2]/span[2]').text

        print(f'Seu ativo é o {ativo}, classificado como {fundo_imobiliario} e seus principais indicadores são os seguintes: ')
        print(f'{ativo} possui {dividend_yield}% de dividend yield dos últimos 12 meses')
        print(f'{ativo} possui um P/PV de {p_pv}')
        print(f'{ativo} possui {numero_cotistas} cotistas atualmente')
        print(f'{ativo} possui {valor_mercado} de valor de mercado')
        print(f'{ativo} possui um total de {valor_caixa} em seu caixa')
        print(f'{ativo} possui um valor patrimonial total de {valor_patrimonio_total}')
        print("    ")
    else:
        acao = navegador.find_element(By.XPATH, '//*[@id="main-header"]/div[2]/div/div[1]/div/ol/li[2]/a/span').text
        dividend_yield = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[4]/div/div[1]/strong').text 
        valorizacao = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[1]/strong').text 
        liquidez_diaria = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[5]/div/div/div[3]/div/div/div').text 
        P_l = navegador.find_element(By.XPATH, '//*[@id="indicators-section"]/div[2]/div/div[1]/div/div[2]/div/div/strong').text 
        ev_ebitda = navegador.find_element(By.XPATH, '//*[@id="indicators-section"]/div[2]/div/div[1]/div/div[5]/div/div/strong').text
        divida_ebitda = navegador.find_element(By.XPATH, '//*[@id="indicators-section"]/div[2]/div/div[2]/div/div[2]/div/div/strong').text
        margem_liquida = navegador.find_element(By.XPATH, '//*[@id="indicators-section"]/div[2]/div/div[3]/div/div[4]/div/div/strong').text
        roe = navegador.find_element(By.XPATH, '//*[@id="indicators-section"]/div[2]/div/div[4]/div/div[1]/div/div/strong').text

        print(f'Seu ativo é o {ativo}, classificado como {acao} e seus principais indicadores são os seguintes: ')
        print(f'{ativo} possui {dividend_yield}% de dividend yield dos últimos 12 meses')
        print(f'{ativo} possui apresentou uma valorização nos últimos 12 meses de {valorizacao}')
        print(f'{ativo} possui uma liquidez diária de {liquidez_diaria}')
        print(f'{ativo} possui {P_l} de Preço sobre lucro (P/L)')
        print(f'{ativo} possui um Enterprise value de {ev_ebitda}')
        print(f'{ativo} possui um índice de endividamento de {divida_ebitda}')
        print(f'{ativo} possui uma margem líquida de {margem_liquida}')
        print(f'{ativo} possui uma ROE de {roe}')
        print("    ")